In [1]:
import nbimporter
import copy
import math
import numpy
import json

import h_make_general_part_layers as mgpl
import h_material_thermal_characteristics as mtc

Importing Jupyter notebook from h_make_general_part_layers.ipynb
Importing Jupyter notebook from h_material_thermal_characteristics.ipynb


# 住宅LV4から住宅負荷計算入力データへのコンバート

## I. 関数(common)

In [2]:
def make_common(common):
    
    return {
        'region': common['region']
    }

## II. 関数(rooms)

### 床下空間の床面積の取得

方位が'Bottom'である床下空間の外皮の部位と、床下空間の土間床を、床下空間の床とみなして、床下空間の床面積を取得する。  
床下空間に属する部位があるが、床下空間の床面積が0㎡の場合、エラーとする。

In [3]:
def get_area_underfloor(envelope):
    
    return sum([ec['area'] for ec in envelope['earthfloor_centers'] if ec['space_type'] == 'underfloor'])

### Roomsの設定

### 1. 容積

$$ V_{space,i} = A_{f,i} h_{space,i} $$

$V_{space,i}$:空間$i$の容積, m<sup>3</sup>  
$A_{f,i}$:空間$i$の床面積, m<sup>2</sup>  
$h_{space,i}$:空間$i$の天井高さ, m  
空間$i$の天井高さ$h_{space,i}$は、空間の種類が「主たる居室」・「その他の居室」・「非居室」の場合は2.4mとし、「床下空間」の場合は0.4mとする。

In [4]:
def get_room_volume(room_type, a_f):
    
    h_space = {
        'main_occupant_room'  : 2.4,
        'other_occupant_room' : 2.4,
        'non_occupant_room'   : 2.4,
        'underfloor'          : 0.4,
    }[room_type]
    
    return a_f * h_space

### 2. 全般換気設備の換気量（外気から）

$$ V_{vent} = A_A h_A N a $$

$$ V_{vent,ex-MR} = V_{vent} \frac{ A_{MR} }{ A_{MR} + A_{OR} } $$

$$ V_{vent,ex-OR} = V_{vent} \frac{ A_{OR} }{ A_{MR} + A_{OR} } $$

$$ V_{vent,ex-NR} = 0 $$

$$ V_{vent,ex-uf} = 0 $$

$V_{vent}$:全般換気設備の換気量, m<sup>3</sup>/h  
$A_A$:床面積の合計, m<sup>2</sup>  
$h_A$:天井高さ, m (=2.4)  
$N$:換気回数, 1/h  
$a$:全般換気設備の換気量の余裕率 (=1.1)  
$V_{vent,ex-MR}$:外気から主たる居室への全般換気設備の換気量, m<sup>3</sup>/h  
$V_{vent,ex-OR}$:外気からその他の居室への全般換気設備の換気量, m<sup>3</sup>/h  
$V_{vent,ex-NR}$:外気から非居室への全般換気設備の換気量, m<sup>3</sup>/h  
$V_{vent,ex-uf}$:外気から床下空間への全般換気設備の換気量, m<sup>3</sup>/h  
$V_{vent,MR-NR}$:主たる居室から非居室への全般換気設備の換気量, m<sup>3</sup>/h  
$V_{vent,OR-NR}$:その他の居室から非居室への全般換気設備の換気量, m<sup>3</sup>/h  

In [5]:
def get_room_vent_from_outdoor(a_a, a_mr, a_or, n):
    
    h_a = 2.4
    a = 1.1
    
    v_vent = a_a * h_a * h_a * n * a
    
    v_vent_ex_mr = v_vent * a_mr / (a_mr + a_or)
    v_vent_ex_or = v_vent * a_or / (a_mr + a_or)
    v_vent_ex_nr = 0.0
    v_vent_ex_uf = 0.0
    
    return v_vent_ex_mr, v_vent_ex_or, v_vent_ex_nr, v_vent_ex_uf

### 3. 全般換気設備の換気量（室間）

$$ V_{vent,MR-NR} = V_{vent,ex-MR} $$

$$ V_{vent,OR-NR} = V_{vent,ex-OR} $$

In [6]:
def get_room_vent_between_rooms(v_vent_ex_mr, v_vent_ex_or):
    
    v_vent_mr_nr = v_vent_ex_mr
    v_vent_or_nr = v_vent_ex_or
    
    return v_vent_mr_nr, v_vent_or_nr

In [7]:
def get_rooms_initial(common, ventilation, envelope):
    
    a_a  = common['total_floor_area'] 
    a_mr = common['main_occupant_room_floor_area']
    a_or = common['other_occupant_room_floor_area'] 
    a_nr = a_a - a_mr - a_or
    a_uf = get_area_underfloor(envelope)
    
    n = ventilation['air_change_rate']
    
    v_vent_ex_mr, v_vent_ex_or, v_vent_ex_nr, v_vent_ex_uf = get_room_vent_from_outdoor(a_a, a_mr, a_or, n)
    v_vent_mr_nr, v_vent_or_nr = get_room_vent_between_rooms(v_vent_ex_mr, v_vent_ex_or)
    
    c_value = 5.0 # ここは住宅構造やUA値等から何らかの設定が必要
    
    def make_room(name, room_type, volume, vent, next_vent, c_value):
        
        return {
            'name'                : name,
            'room_type'           : room_type,
            'volume'              : volume,
            'vent'                : vent,
            'next_vent'           : next_vent,
            'c_value'             : c_value,
            'CrossVentRoom'       : False,
            'RadHeat'             : False,
            'Beta'                : 0.0,
            'LinearHeatBridge'    : [],
            'EarthfloorPerimeter' : [],
            'surface'             : [],
        }
    
    rooms = []

    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積が0m2になっています。')
        
    room_mr = make_room(
        name='main_occupant_room',
        room_type='main_occupant_room',
        volume=get_room_volume(room_type='main_occupant_room', a_f=a_mr),
        vent=v_vent_ex_mr,
        next_vent=[],
        c_value=c_value
    )
    
    rooms.append(room_mr)
    
    if a_or > 0.0:
        
        room_or = make_room(
            name='other_occupant_room',
            room_type='other_occupant_room',
            volume=get_room_volume(room_type='other_occupant_room', a_f=a_or),
            vent=v_vent_ex_or,
            next_vent=[],
            c_value=c_value
        )
        
        rooms.append(room_or)
        
    if a_nr > 0.0:
        
        next_vent = []
        
        if a_mr > 0.0:
            next_vent.append({
                'upstream_room_name' : 'main',
                'volume'             : v_vent_mr_nr,
            })
        
        if a_or > 0.0:
            next_vent.append({
                'upstream_room_name': 'other',
                'volume': v_vent_or_nr,
            })
        
        room_nr = make_room(
            name='non_occupant_room',
            room_type='non_occupant_room',
            volume=get_room_volume(room_type='non_occupant_room', a_f=a_nr),
            vent=v_vent_ex_nr,
            next_vent=next_vent,
            c_value=c_value
        )
        
        rooms.append(room_nr)
        
    if a_uf > 0.0:
        
        room_uf = make_room(
            name='underfloor',
            room_type='underfloor',
            volume=get_room_volume(room_type='underfloor', a_f=a_uf),
            vent=v_vent_ex_uf,
            next_vent=[],
            c_value=c_value
        )
        
        rooms.append(room_uf)
    
    return rooms

### 表面熱伝達抵抗の取得

#### Outline

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  
　　　　　付録 A 住宅の平均熱貫流率算出に用いる建材等の熱物性値等   
　　　　　　表 3.1 表面熱伝達抵抗   
　　　　　　表 3.2 表面熱伝達抵抗（界壁・界床の場合） 

### Boundaryの設定

#### Outline

方位情報より方位角・傾斜角を取得する。

#### Function

In [8]:
def get_directionangle_from_direction(direction):
    
    Direction_to_DirectionAngle = {
        'Top': None,
        'N':   180,
        'NE': -135,
        'E':   -90,
        'SE':  -45,
        'S':     0,
        'SW':   45,
        'W':    90,
        'NW':  135,
        'Bottom':          None,
        'ClosedSpace':     None,
        'OpenBackFloor':   None,
        'ResidenceSpace':  None,
        'ClosedBackFloor': None
    }
    
    return Direction_to_DirectionAngle[direction]

In [9]:
def get_inclinationangle_from_direction(direction):
    
    Direction_to_InclinationAngle = {
        'Top': 0,
        'N':  90,
        'NE': 90,
        'E':  90,
        'SE': 90,
        'S':  90,
        'SW': 90,
        'W':  90,
        'NW': 90,
        'Bottom': 180,
        'ClosedSpace':     None,
        'OpenBackFloor':   None,
        'ResidenceSpace':  None,
        'ClosedBackFloor': None
    }
    
    return Direction_to_InclinationAngle[direction]

#### Outline

方位情報より隣室温度差係数を取得する。

#### Function

In [10]:
def get_tempdifferentfactor_from_direction(region, direction):
    
    Direction_to_NextSpace = {
        'Top':     'outside',
        'N':       'outside',
        'NE':      'outside',
        'E':       'outside',
        'SE':      'outside',
        'S':       'outside',
        'SW':      'outside',
        'W':       'outside',
        'NW':      'outside',
        'Bottom':  'outside',
        'ClosedSpace':     'underfloor',
        'OpenBackFloor':   'underfloor',
        'ResidenceSpace':  'next_room',
        'ClosedBackFloor': 'next_room'
    }    
    
    NextSpace_to_TempDifferentFactor = {
        # 外気又は外気に通じる空間(小屋裏・天井裏・共用部・屋内駐車場・メーターボックス・エレベーターシャフト等)
        'outside'    : {1: 1.00, 2: 1.00, 3: 1.00, 4: 1.00, 5: 1.00, 6: 1.00, 7: 1.00, 8: 1.00 }, 
        # 外気に通じていない空間(昇降機室、共用機械室、倉庫等)又は外気に通じる床裏
        'underfloor' : {1: 0.70, 2: 0.70, 3: 0.70, 4: 0.70, 5: 0.70, 6: 0.70, 7: 0.70, 8: 0.70 }, 
        # 住戸、住戸と同様の熱的環境の空問(空調された共用部等)又は外気に通じていない床裏(ピット等）
        'next_room'  : {1: 0.05, 2: 0.05, 3: 0.05, 4: 0.15, 5: 0.15, 6: 0.15, 7: 0.15, 8: 0.15 }  
    }
    
    return  NextSpace_to_TempDifferentFactor[Direction_to_NextSpace[direction]][region]

#### Outline

Boundaryを設定する。

#### Function

In [11]:
def get_boundary_for_outer_skin(region, nextspace, direction):
    
    Type = {
        'Top':     'Outdoor',
        'N':       'Outdoor',
        'NE':      'Outdoor',
        'E':       'Outdoor',
        'SE':      'Outdoor',
        'S':       'Outdoor',
        'SW':      'Outdoor',
        'W':       'Outdoor',
        'NW':      'Outdoor',
        'Bottom':  'Outdoor',
        'ClosedSpace':     'DeltaTCoeff',
        'OpenBackFloor':   'DeltaTCoeff',
        'ResidenceSpace':  'DeltaTCoeff',
        'ClosedBackFloor': 'DeltaTCoeff'
    }    
    
    boundary = {
        'Outdoor': {
            'name': direction,
            'Type': 'Outdoor',
            'DirectionAngle': get_directionangle_from_direction(direction if nextspace=='Outdoor' else nextspace),
            'InclinationAngle': get_inclinationangle_from_direction(direction if nextspace=='Outdoor' else nextspace),
            'GroundReflectRate': 0.1
        },
        'DeltaTCoeff': {
            'name': direction,
            'Type': 'DeltaTCoeff',
            'TempDifferFactor': get_tempdifferentfactor_from_direction(region, direction if nextspace =='Outdoor' else nextspace)
        }
    }[Type[direction if nextspace=='Outdoor' else nextspace]]

    return boundary

In [12]:
def get_boundary_for_inner_skin(type, nextspace):
    
    boundary = {
        'name': 'NextRoom' + '_' + nextspace,
        'Type': 'NextRoom',
        'room_type': nextspace
    }
    
    return boundary

In [13]:
def get_boundary_for_earthfloorperimeter(region, direction):
    
    Type = {
        'Top':     'Outdoor',
        'N':       'Outdoor',
        'NE':      'Outdoor',
        'E':       'Outdoor',
        'SE':      'Outdoor',
        'S':       'Outdoor',
        'SW':      'Outdoor',
        'W':       'Outdoor',
        'NW':      'Outdoor',
        'Bottom':  'Outdoor',
        'ClosedSpace':     'DeltaTCoeff',
        'OpenBackFloor':   'DeltaTCoeff',
        'ResidenceSpace':  'DeltaTCoeff',
        'ClosedBackFloor': 'DeltaTCoeff'
    }    
    
    boundary = {
        'Outdoor': {
            'name': direction,
            'Type': 'Outdoor'
        },
        'DeltaTCoeff': {
            'name': direction,
            'Type': 'DeltaTCoeff',
            'TempDifferFactor': get_tempdifferentfactor_from_direction(region, direction)
        }
    }[Type[direction]]

    return boundary

### directionの設定

#### Outline

方位（Top/N/NE/E/SE/S/SW/W/NW/Bottom）より方位（Upward/Vertical/Downward）を決定する。  
方位（Top/N/NE/E/SE/S/SW/W/NW/Bottom）にClosedSpace/OpenBackFloor/ResidenceSpace/ClosedBackFloorが含まれるが、方位と温度差係数を別の変数として扱うべき。

#### Function

In [14]:
def get_direction_from_direction_for_outer_skin(direction):
    
    return {
        'Top':     'Downward',
        'N':       'Vertical',
        'NE':      'Vertical',
        'E':       'Vertical',
        'SE':      'Vertical',
        'S':       'Vertical',
        'SW':      'Vertical',
        'W':       'Vertical',
        'NW':      'Vertical',
        'Bottom':  'Upward',
        'ClosedSpace':     'Vertical', # 暫定的な処理。方位と温度差係数を別の変数として扱うべき。
        'OpenBackFloor':   'Upward',    # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。
        'ResidenceSpace':  'Vertical', # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。
        'ClosedBackFloor': 'Upward'     # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。             
    }[direction]

#### Outline

内壁種類（InnerCeiling/InnerWall/InnerFloo/GroundFloor）と室より方位（Upward/Vertical/Downward）を決定する。

#### Function

In [15]:
def get_direction_from_direction_for_inner_skin(space, type):
    
    return {
        'main_occupant_room': {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'other_occupant_room':      {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'non_occupant_room':  {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'underfloor': {'InnerCeiling': None,       'InnerWall': None,        'InnerFloor': None,     'GroundFloor': 'Downward'}
    }[space][type]

### 層構成の設定

#### Outline

容積比熱情報がない場合は、0kJ/(㎥・K)とする。

#### Function

In [16]:
def make_layer(name, thick, cond, specH, **kwargs) :
    
    return {
            'name'  : name,
            'thick' : thick,
            'cond'  : cond,
            'specH' : 0 if specH == None else specH
    }

### 詳細計算法の場合の壁体の設定

#### Outline

Partsの複数の要素を複数の部位に分離する。

#### Function

In [17]:
def make_d_wall_for_inputalldetails(d_wall):

    d_walls = []
    for x in d_wall['Parts'] :
        d_walls.append({
            'name':      d_wall['name'],
            'nextspace': d_wall['nextspace'],
            'direction': d_wall['direction'],
            'area':      d_wall['area']*x['AreaRatio'],
            'space':     d_wall['space'],
            'type':      d_wall['type'],
            'structure':     d_wall['structure'],        
            'IsSunshadeInput': d_wall['IsSunshadeInput'],
            'Layers': [make_layer(**layer) for layer in x['Layers']]
        })
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
    
    return d_walls

### 簡略計算方法①：面積比率法（充填断熱する場合又は充填断熱し付加断熱する場合）の場合の壁体の設定 

#### Outline

Partsの複数の要素を複数の部位に分離する。  
面積比率は、木造住宅の建て方及び構法／工法の種類等に応じて与えられる。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### Outline

<div style="text-align: center;"> 表3-1 木造における床の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・床梁工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（大引間に断熱）  | 0.85 | 0.15 |
| 軸組構法・束立大引工法（根太間及び大引間に断熱）  |  |  |
| 軸組構法・剛床工法  | 0.85 | 0.15 |
| 軸組構法・床梁土台同面工法（根太間に断熱）  | 0.70 | 0.30 |
| 枠組壁工法（根太間に断熱）  | 0.87 | 0.13 |

<div style="text-align: center;"> 表3-2 軸組構法の束立大引工法において根太間及び大引間に断熱する場合の床の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（根太間断熱材＋大引間断熱材） | 断熱部分＋熱橋部分<br>（根太間断熱材＋大引材等） | 断熱部分＋熱橋部分<br>（根太材＋大引間断熱材） | 熱橋部分<br>（根太材＋大引材等） |
|---|---|---|---|---|
|  軸組構法の束立大引工法において<br>根太間及び大引間に断熱する場合 | 0.72 | 0.12 | 0.13 | 0.03 |

#### Function

In [18]:
def get_area_ratio_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        area_ratio_wood_floor = {
            'InsulJoitandsleeper' : 0.72,
            'InsuljoistHeatbridgesleeper' : 0.12,
            'InsulsleeperHeatbridgejoist' : 0.13,
            'HeatbridgeJoistandsleeper' : 0.03
        }
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.70, 'Heatbridge' : 0.30}
    elif FloorConstructionMethod == 'WallInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.87, 'Heatbridge' : 0.13}
    else :
        raise ValueError
    
    return area_ratio_wood_floor

#### Outline

<div style="text-align: center;"> 表4-1 木造における外壁（界壁）の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・柱、間柱間に断熱 | 0.83 | 0.17 |
| 軸組構法・柱、間柱間に断熱し付加断熱 |  |  |
| 枠組壁工法・たて枠間に断熱 | 0.77 | 0.23 |
| 枠組壁工法・たて枠間に断熱＋付加断熱 |  |  |

<div style="text-align: center;"> 表4-2 軸組構法において柱・間柱間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（柱・間柱間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（柱・間柱間断熱材＋付加断熱層内熱橋部分） | 断熱部分＋熱橋部分<br>（構造部材等＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部分） |
|---|---|---|---|---|
| 付加断熱層内熱橋<br>部分が「横下地」 | 0.75 | 0.08 | 0.12 | 0.05 |
| 付加断熱層内熱橋<br>部分が「縦下地」 | 0.79 | 0.04 | 0.04 | 0.13 |

<div style="text-align: center;"> 表4-3 枠組壁工法においてたて枠間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（充填断熱材＋付加断熱材） | 断熱部分＋熱橋部<br>（充填断熱材＋付加断熱層内熱橋部） | 断熱部分＋熱橋部<br>（構造部材等＋付加断熱材） | 断熱部分＋熱橋部<br>（まぐさ＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部） | 熱橋部分<br>（まぐさ＋付加断熱層内熱橋部） |
|---|---|---|---|---|---|---|
| 付加断熱層内熱橋部分が「横下地」 | 0.69 | 0.08 | 0.14 | 0.02 | 0.06 | 0.01 |
| 付加断熱層内熱橋部分が「縦下地」 | 0.76 | 0.01 | 　 | 0.02 | 0.20 | 0.01 |

#### Function

In [19]:
def get_area_ratio_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'InsulColumnandadd' : 0.75,
            'InsulcolumnHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.05
        }
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        area_ratio_wood_wall = {
            'InsulColumnandadd' : 0.79,
            'InsulcolumnHeatbridgeadd' : 0.04,
            'InsuladdHeatbridgemember' : 0.04,
            'HeatbridgeMemberandadd' : 0.13
        }
    elif WallConstructionMethod == 'WallInsuljamb' :
        area_ratio_wood_wall = {'Insulation' : 0.77, 'Heatbridge' : 0.23}
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'InsulFilledandadd' : 0.69,
            'InsulfilledHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.14,
            'InsuladdHeatbridgelintel' : 0.02,
            'HeatbridgeMemberandadd' : 0.06,
            'HeatbridgeLintelandadd' : 0.01
        }
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        area_ratio_wood_wall = {        
            'InsulFilledandadd' : 0.76,
            'InsulfilledHeatbridgeadd' : 0.01,
            'InsuladdHeatbridgemember' : 0.00,
            'InsuladdHeatbridgelintel' : 0.02,
            'HeatbridgeMemberandadd' : 0.20,
            'HeatbridgeLintelandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_wall

#### Outline

<div style="text-align: center;"> 表5 木造における天井の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 桁・梁間に断熱する場合 | 0.87 | 0.13 |

#### Function

In [20]:
def get_area_ratio_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    else :
        raise ValueError

    return area_ratio_wood_ceiling

#### Outline

<div style="text-align: center;"> 表6-1 木造における屋根の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| たるき間に断熱する場合  | 0.86 | 0.14 |

<div style="text-align: center;"> 表6-2 木造においてたるき間に断熱し付加断熱（横下地）する場合の屋根の面積比率  </div>

| 工法の種類等 | 断熱部分<br>（たる木間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（たる木間断熱材＋付加断熱層内熱橋部（下地たる木）） | 断熱部分＋熱橋部分<br>（構造部材＋付加断熱材） | 熱橋部分<br>（構造部材＋付加断熱層内熱橋部（下地たる木）） |
|---|---|---|---|---|
| たるき間に断熱し付加断熱（横下地）する場合  | 0.79 | 0.08 | 0.12 | 0.01 |

#### Function

In [21]:
def get_area_ratio_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        area_ratio_wood_roof = {'Insulation' : 0.86, 'Heatbridge' : 0.14}
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        area_ratio_wood_roof = {
            'InsulRafterandadd' : 0.79,
            'InsulrafterHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_roof

#### Function

In [22]:
def get_area_ratio_wood(d_wall):
    
    if d_wall['Type'] == 'Roof' :
        return get_area_ratio_wood_roof(d_wall['RoofConstructionMethod'])
    elif d_wall['Type'] == 'Ceiling' or d_wall['Type'] == 'BoundaryCeiling' :
        return get_area_ratio_wood_ceiling(d_wall['CeilingConstructionMethod'])        
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        return get_area_ratio_wood_wall(d_wall['WallConstructionMethod'])            
    elif d_wall['Type'] == 'Floor' :
        return get_area_ratio_wood_floor(d_wall['FloorConstructionMethod'])    
    else :
        raise ValueError        

In [23]:
def make_d_wall_for_inputalllayers(d_wall):
    
    area_ratio = get_area_ratio_wood(d_wall)

    d_walls = []
    for x in d_wall['Parts'] :
        d_walls.append({
            'name':      d_wall['name'],
            'nextspace': d_wall['nextspace'],
            'direction': d_wall['direction'],
            'area':      d_wall['area']*area_ratio[x],
            'space':     d_wall['space'],
            'type':      d_wall['type'],
            'structure':     d_wall['structure'],        
            'IsSunshadeInput': d_wall['IsSunshadeInput'],
            'Layers': [make_layer(**layer) for layer in x['Layers']]
        })
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
    
    return d_walls

### 簡略計算方法②：熱貫流率補正法、鉄骨造の場合の壁体の設定 

#### Outline

補正熱貫流率より部位の平均熱貫流率を算出する。  
部位の平均熱貫流率に一致するよう、一般部の層構成を調整する。

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### Outline

<div style="text-align: center;"> 表7 木造における一般部位の断熱工法等に応じた補正熱貫流率 </div>

| 部位 | 断熱工法等 | 軸組構法補正熱貫流率 | 枠組壁構法補正熱貫流率 |
|---|---|---|---|
| 床 | 　 | 0.13 | 0.08 |
| 上階側界床 | 　 | 0.13 | 0.08 |
| 下階側界床 | 　 | 0.13 | 0.08 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）する場合 | 0.09 | 0.13 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）し付加断熱する場合 | 0.09 | 0.13 |

| 部位 | 断熱工法等 | 補正熱貫流率 |
|---|---|---|
| 外壁・界壁 | 土壁で外張断熱の場合 | 0.04 |
| 外壁・界壁 | 土壁以外で外張断熱の場合 | 0.02 |
| 天井 | 桁・梁間に断熱する場合 | 0.05 |
| 屋根 | 充填断熱（たるき間に断熱）する場合 | 0.11 |
| 屋根 | 充填断熱（たるき間に断熱）し付加断熱する場合 | 0.11 |
| 屋根 | 外張断熱工法 | 0.02 |

#### Function

In [24]:
def get_Ur_wood(d_wall):
    # Type : 部位種別
    # ConstructionMethod : 工法種類
    # 'FrameInsulcolumn'（軸組構法）/'WallInsul'（枠組壁構法）
    
    if d_wall['Type'] == 'Floor' or d_wall['Type'] == 'BoundaryCeiling' or d_wall['Type'] == 'BoundaryFloor' :
        if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
            Ur = 0.13
        elif d_wall['ConstructionMethod'] == 'WallInsul' :
            Ur = 0.08
        else :
            raise ValueError

    # WallInsulationMethod : 壁の断熱種類
    # 'InsulColumn'（充填断熱（柱・間柱間に断熱））/'InsulColumnandadd'（充填断熱（柱・間柱間に断熱）+付加断熱）/
    # 'OuterInsulMudWall'（土壁で外張断熱）/'OuterInsul'（土壁以外で外張断熱）    
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        if d_wall['WallInsulationMethod'] == 'InsulColumn' or d_wall['RoofInsulationMethod'] == 'InsulColumnandadd' :
            if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
                Ur = 0.09
            elif d_wall['ConstructionMethod'] == 'WallInsul' :
                Ur = 0.13
            else :
                raise ValueError
        elif d_wall['WallInsulationMethod'] == 'OuterInsulMudWall' :
            Ur = 0.04
        elif d_wall['WallInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    elif d_wall['Type'] == 'Ceiling' :
        Ur = 0.05

    # RoofInsulationMethod : 屋根の断熱種類
    # 'InsulRafter'（充填断熱（たるき間に断熱））/'InsulRafterandadd'（充填断熱（たるき間に断熱）+付加断熱）/'OuterInsul'（外張断熱工法）
    elif d_wall['Type'] == 'Roof' :
        if d_wall['RoofInsulationMethod'] == 'InsulRafter' or d_wall['RoofInsulationMethod'] == 'InsulRafterandadd' :
            Ur = 0.11
        elif d_wall['RoofInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    else :
        raise ValueError
    
    return Ur

#### Outline

<div style="text-align: center;"> 表8 鉄骨造における一般部位の熱橋部分（柱及び梁以外）の仕様に応じた補正熱貫流率 </div>

| 「外装材＋断熱補強材」の熱抵抗(m<sup>2</sup>･K/W) | 補正熱貫流率ܷ |
|---|---|
| 1.7以上 | 0.00 |
| 1.7未満1.5以上 | 0.10 |
| 1.5未満1.3以上 | 0.13 |
| 1.3未満1.1以上 | 0.14 |
| 1.1未満0.9以上 | 0.18 |
| 0.9未満0.7以上 | 0.22 |
| 0.7未満0.5以上 | 0.40 |
| 0.5未満0.3以上 | 0.45 |
| 0.3未満0.1以上 | 0.60 |
| 0.1未満 | 0.70 |

#### Function

In [25]:
def get_Ur_steel(Rc):
    # Rer : 「外装材＋断熱補強材」の熱抵抗, m2K/W
    
    if Rc >= 1.7 :
        Ur = 0.00
    elif Rc >= 1.5 :
        Ur = 0.10
    elif Rc >= 1.3 :
        Ur = 0.13    
    elif Rc >= 1.1 :
        Ur = 0.14    
    elif Rc >= 0.9 :
        Ur = 0.18    
    elif Rc >= 0.7 :
        Ur = 0.22    
    elif Rc >= 0.5 :
        Ur = 0.40    
    elif Rc >= 0.3 :
        Ur = 0.45    
    elif Rc >= 0.1 :
        Ur = 0.60  
    elif Rc < 0.1 :
        Ur = 0.70
    else :
        raise ValueError

    return Ur

#### Outline

$$
\displaystyle
U_{i}= \frac{1}{R_{g,i}} + U_{r,i}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$R_{g,i}$: 一般部位$i$の断熱部分の熱抵抗, m<sup>2</sup>K/W  
$U_{r,i}$: 一般部位$i$の補正熱貫流率, W/(m<sup>2</sup>・K)  

#### Function

In [26]:
def calc_UA_from_Ur(d_wall):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro = mtc.get_surface_thermal_resistance(general_part_type=d_wall['general_part_type'], next_space=d_wall['next_space'])
    Ri = get_InHetaTrans(d_wall['Type'])
    Rg = Ro + sum(x['Thick']/x['Cond'] for x in d_wall['Parts']['Insulation']['Layers']) + Ri
    
    # 補正熱貫流率
    if d_wall['Structure'] == 'Wood' :
        Ur = get_Ur_wood(d_wall)
    elif d_wall['Structure'] == 'Steel' :
        Ur = get_Ur_steel(d_wall['RCladding'])
    else :
        raise ValueError
    
    return 1/Rg + Ur

#### Outline

一般部位の熱貫流率$U_A$に一致するよう、一般部位の部分の層構成を調整する。  
－$U_A$に一致するまで、Layersの順番に厚を減らす  
－$U_A$に一致するまで、Layersの逆順に厚を減らす  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの順番）  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順）  

#### Function

In [27]:
# UAをもとに層構成を調整
# Layersの順番に厚を減らす
def get_layers_from_UA_acending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in layers :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [28]:
# UAをもとに層構成を調整
# Layersの逆順に厚を減らす
def get_layers_from_UA_descending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in reversed(layers) :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [29]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
def get_layers_from_UA_ins_acending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(0,len(l_copy)) :
                if cond_temp == l_copy[y]['Cond'] :
                    sort_list.append(y)
                    l_copy[y]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

In [30]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
def get_layers_from_UA_ins_descending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(len(l_copy),0,-1) :
                if cond_temp == l_copy[y-1]['Cond'] :
                    sort_list.append(y-1)
                    l_copy[y-1]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

In [31]:
def make_d_wall_for_inputUR(d_wall_init):
    
    r_i, r_o = mtc.get_surface_thermal_resistance(general_part_type=d_wall_init['general_part'], next_space=d_wall_init['next_space'])
    
    UA = calc_UA_from_Ur(d_wall_init)
    
    d_wall = [{
        'name':      d_wall['name'],
        'nextspace': d_wall['nextspace'],
        'direction': d_wall['direction'],
        'area':      d_wall['area'],
        'space':     d_wall['space'],
        'type':      d_wall['type'],
        'structure':     d_wall['structure'],        
        'IsSunshadeInput': d_wall['IsSunshadeInput'],
        'Layers': get_layers_from_UA_ins_descending(d_wall_init['Parts'][0]['Layers'], UA, 1/r_o, 1/r_i)
    }]
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
            
    return d_wall

### RC造の場合の壁体の設定

#### Outline

情報を維持する。

#### Function

In [32]:
def make_d_wall_for_inputRC(d_wall_init):
    
    d_wall = [{
        'name':      d_wall['name'],
        'nextspace': d_wall['nextspace'],
        'direction': d_wall['direction'],
        'area':      d_wall['area'],
        'space':     d_wall['space'],
        'type':      d_wall['type'],
        'structure':     d_wall['structure'],        
        'IsSunshadeInput': d_wall['IsSunshadeInput'],
        'Layers':  [make_layer(**layer) for layer in d_wall['Parts'][0]['Layers']]
    }]
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
            
    return d_wall

### Walls要素をRoomsに展開

#### Outline

Partsの複数の要素を複数の部位に分離する。

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [33]:
def integrate_walls_to_rooms(region, d_walls_init, d_rooms):
    
    d_walls = []
    for x in d_walls_init :
        # 直接入力
        if x['InputMethod'] == 'InputUA':
            d_walls['layers'] = mgpl.get_layers(general_part_type=x['type'],next_space=x['nextspace'], structure='wood', u_value=x['UA'])
        # 木造 詳細計算法
        elif  x['structure'] == 'wood' and x['InputMethod'] == 'InputAllDetails' :
            d_walls.extend(make_d_wall_for_inputalldetails(x))
        # 木造 簡略計算法①
        elif  x['structure'] == 'wood' and x['InputMethod'] == 'InputAllLayers' :
            d_walls.extend(make_d_wall_for_inputalllayers(x))
        # 木造 簡略計算法②、鉄骨造
        elif (x['structure'] == 'wood' and x['InputMethod'] == 'InputUR') or x['structure'] == 'Steel' :
            d_walls.extend(make_d_wall_for_inputUR(x))
        # RC造
        elif x['structure'] == 'RC' :
            d_walls.extend(make_d_wall_for_inputRC(x))       
        # その他
        elif x['structure'] == 'other' :
            raise ValueError        
        else :
            raise ValueError

    n = {'Roof': 1, 'Ceiling': 1, 'Wall': 1, 'Floor': 1, 'BoundaryCeiling': 1, 'BoundaryWall': 1, 'BoundaryFloor': 1}
    
    for gp in d_walls :
        
        r_i, r_o = mtc.get_surface_thermal_resistance(general_part_type=gp['general_part_type'], next_space=gp['next_space'])
        
        for y in d_rooms :
            
            if gp['space_type'] == y['room_type']:
                
                y['surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(gp['direction'] if gp['nextspace']=='Outdoor' else gp['nextspace']),
                    'floor': True if gp['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, gp['nextspace'], gp['direction']),
                    'unsteady': True, 
                    'IsSoil': False,
                    'name': gp['type'] + str(n[gp['type']]), 
                    'area': gp['area'], 
                    'flr' : 0,
                    'Wall': {
                        'OutEmissiv': 0.9,
                        'OutSolarAbs': 0.8,
                        'OutHeatTrans': 1/r_o,
                        'InHeatTrans': 1/r_i,
                        'Layers':gp['Layers']
                    },
                    'sunbreak':{'D': gp['Z'], 'WI1': 500, 'WI2': 500, 'hi': gp['Y1'], 'WR': 0, 'WH': gp['Y2'], 
                                'name': 'ひさし'} if gp['IsSunshadeInput'] == True else {}
                })
                n[gp['type']] = n[gp['type']] + 1
    
    return d_rooms

### InnerWalls要素をRoomsに展開

#### Outline

床下空間上の床、室上の床を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [34]:
def integrate_innerwalls_to_rooms(region, d_innerwalls, d_rooms):
    
    n = {'InnerCeiling': 1, 'InnerWall': 1, 'InnerFloor': 1, 'GroundFloor': 1}
    for x in d_innerwalls :
        
        r_i, r_o = mtc.get_surface_thermal_resistance(general_part_type=x['general_part'], next_space=x['next_space'])
        
        for y in d_rooms :
            if x['space'] == y['room_type'] :    
                y['Surface'].append({
                    'skin': False, 
                    'direction': get_direction_from_direction_for_inner_skin(x['space'], x['type']),
                    'floor': True if (x['type'] == 'GroundFloor' or x['type'] == 'InnerFloor') else False,
                    'boundary': get_boundary_for_inner_skin(x['type'], x['nextspace']),
                    'unsteady': True,
                    'IsSoil': False,
                    'name': x['type'] + str(n[x['type']]), 
                    'area': x['area'], 
                    'flr' : 0,
                    'Wall': {
                        'OutHeatTrans': 1/r_o,
                        'InHeatTrans': 1/r_i,
                        'Layers':x['Layers']
                    },
                })
                n[x['type']] = n[x['type']] + 1
    
    return d_rooms

### Doors要素をRoomsに展開

#### Outline

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [35]:
def integrate_doors_to_rooms(region, d_doors, d_rooms):
    
    n = 1
    for x in d_doors :
        for y in d_rooms :
            if x['space'] == y['room_type'] :
                eta = 0.034*x['U']
                y['Surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(x['direction']),
                    'floor': True if x['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, x['nextspace'], x['direction']),
                    'unsteady': False, 
                    'name': 'Door' + str(n), 
                    'area': x['area'], 
                    'flr' : 0,
                    'Window': {
                        'Eta': eta,
                        'SolarTrans': eta,
                        'SolarAbsorp': 0,
                        'Uw': x['U'],
                        'OutEmissiv': 0.9,
                        'OutHeatTrans': 1/0.04,
                        'InHeatTrans': 1/0.11,
                    },
                    'sunbreak':{'D': x['Z'], 'WI1': 500, 'WI2': 500, 'hi': x['Y1'], 'WR': 0, 'WH': x['Y2'], 
                                'name': 'ひさし'} if x['IsSunshadeInput'] == True else {}
                })
                n = n + 1
    
    return d_rooms

### Windows要素をRoomsに展開

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第四節　日射熱取得率  

#### Outline

<div style="text-align: center;"> 付録C　表1 ガラスの垂直面日射熱取得率 </div>

| ガラスの仕様 | 　 | 　 | 付属部材なし | 和障子 | 外付けブラインド |
|---|---|---|---|---|---|
| 三層複層 | 2枚以上のガラス表面にLow-E膜を使用したLow-E三層複層ガラス | 日射取得型 | 0.54 | 0.34 | 0.12 |
| 三層複層 | 2枚以上のガラス表面にLow-E膜を使用したLow-E三層複層ガラス | 日射遮蔽型 | 0.33 | 0.22 | 0.08 |
| 三層複層 | Low-E三層複層ガラス | 日射取得型 | 0.59 | 0.37 | 0.14 |
| 三層複層 | Low-E三層複層ガラス | 日射遮蔽型 | 0.37 | 0.25 | 0.10 |
| 三層複層 | 三層複層ガラス　| 　 | 0.72 | 0.38 | 0.18 |
| 二層複層 | Low-E二複層ガラス | 日射取得型 | 0.64 | 0.38 | 0.15 |
| 二層複層 | Low-E二複層ガラス | 日射遮蔽型 | 0.40 | 0.26 | 0.11 |
| 二層複層 | 二層複層ガラス | 　 | 0.79 | 0.38 | 0.17 |
| 二層複層 | 単板ガラス2枚を組み合わせたもの | 日射遮蔽型 | 0.79 | 0.38 | 0.17 |
| 単層 | 単板ガラス　| 　 | 0.88 | 0.38 | 0.19 |

In [36]:
def get_eta_from_specification(typeFrame, typeGlass, typeShade):
    
    f_frame = {'WoodOrResin': 0.72, 'Steel': 0.8}[typeFrame]
    
    eta = {
        '3WgG':            {'None': 0.54, 'Shoji': 0.34, 'ExtarnalBlind': 0.12}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス2枚）日射取得型
        '3WsG':            {'None': 0.33, 'Shoji': 0.22, 'ExtarnalBlind': 0.08}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス2枚）日射遮蔽型
        '3LgG':            {'None': 0.59, 'Shoji': 0.37, 'ExtarnalBlind': 0.14}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス1枚）日射取得型
        '3LsG':            {'None': 0.37, 'Shoji': 0.25, 'ExtarnalBlind': 0.10}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス1枚）日射遮蔽型
        '2LgG':            {'None': 0.64, 'Shoji': 0.38, 'ExtarnalBlind': 0.15}, # 二層複層 Low-E複層ガラス日射取得型
        '2LsG':            {'None': 0.40, 'Shoji': 0.26, 'ExtarnalBlind': 0.11}, # 二層複層 Low-E複層ガラス日射遮蔽型
        '2FAheatreflect1': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス1種
        '2FAheatreflect2': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス2種
        '2FAheatreflect3': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス3種
        '2FAheatabsorb2':  {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線吸収板ガラス2種
        '2FAmulti':        {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 複層ガラス
        '2FAsingle':       {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 単板ガラス2枚
        'Theatreflect1':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス1種
        'Theatreflect2':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス2種
        'Theatreflect3':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス3種
        'Theatabsorb2':    {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線吸収板ガラス2種
        'Tsingle':         {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}  # 単層 単板ガラスその他
    }[typeGlass][typeShade]
    
    return eta*f_frame

In [37]:
def get_eta(d_window):
    
    if d_window['EtaInputMethod'] == 'InputValue' :
        if d_window['TypeWindow'] == 'Single' :
            eta = d_window['Eta']
        if d_window['TypeWindow'] == 'Double' :
            eta = d_window['EtaInside']*d_window['EtaOutside']*1.06/ \
                  (0.72 if d_window['TypeFrameInside'] == 'WoodOrResin' and d_window['TypeFrameOutside'] == 'WoodOrResin' else 0.8)
    elif d_window['EtaInputMethod'] == 'InputSpecification' :
        if d_window['TypeWindow'] == 'Single' :
            eta = get_eta_from_specification(d_window['TypeFrame'],d_window['TypeGlass'],d_window['TypeShade'])
        if d_window['TypeWindow'] == 'Double' :
            eta = get_eta_from_specification(d_window['TypeFrameInside'],d_window['TypeGlassInside'],d_window['TypeShadeInside'])* \
                  get_eta_from_specification(d_window['TypeFrameOutside'],d_window['TypeGlassOutside'],d_window['TypeShadeOutside'])* \
                  1.06/(0.72 if d_window['TypeFrameInside'] == 'WoodOrResin' and d_window['TypeFrameOutside'] == 'WoodOrResin' else 0.8)
    else :
        raise ValueError
    
    return eta

### Windows要素をRoomsに展開

#### Outline

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [38]:
def integrate_windows_to_rooms(region, d_windows, d_rooms):
    
    n = 1
    for x in d_windows :
        for y in d_rooms :
            if x['space'] == y['room_type'] :
                eta = get_eta(x)
                y['Surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(x['direction']),
                    'floor': True if x['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, x['nextspace'], x['direction']),
                    'unsteady': False, 
                    'name': 'Window' + str(n), 
                    'area': x['area'], 
                    'flr' : 0,
                    'Window': {
                        'Eta': eta,
                        'SolarTrans': eta,
                        'SolarAbsorp': 0,
                        'UW': x['UW'],
                        'OutEmissiv': 0.9,
                        'OutHeatTrans': 1/0.04,
                        'InHeatTrans': 1/0.11,
                    },
                    'sunbreak':{'D': x['Z'], 'WI1': 500, 'WI2': 500, 'hi': x['Y1'], 'WR': 0, 'WH': x['Y2'], 
                                'name': 'ひさし'} if x['IsSunshadeInput'] == True else {}
                })
                n = n + 1
    
    return d_rooms

### Heabridges要素をRoomsに展開

#### Function

In [39]:
def integrate_heatbridges_to_rooms(region, d_heatbridges, d_rooms):
    
    n = 1
    for x in d_heatbridges :
        for y in d_rooms :
            if x['space'] == y['room_type'] :
                y['LinearHeatBridge'].append({
                    'name': 'Heatbridge' + str(n), 
                    'LinearHeatTrans': x['psi'], 
                    'LinearHeatTransLength': x['length'],
                    'boundary1': get_boundary_for_outer_skin(region, x['nextspace1'], x['direction1']),
                    'boundary2': get_boundary_for_outer_skin(region, x['nextspace2'], x['direction2'])
                })
                n = n + 1
    
    return d_room

### EarthfloorPerimeters要素をRoomsに展開

#### Function

In [40]:
def integrate_earthfloorperimeters_to_rooms(region, d_earthfloorperimeters, d_rooms):
    
    n = 1
    for x in d_earthfloorperimeters :
        for y in d_rooms :
            if x['space'] == y['room_type'] :
                y['EarthfloorPerimeter'].append({
                    'name': 'EarthfloorPerimeter' + str(n), 
                    'LinearHeatTrans': x['psi'], 
                    'LinearHeatTransLength': x['length'],
                    'boundary': get_boundary_for_earthfloorperimeter(region, 
                                                                     x['direction'] if x['nextspace']=='Outdoor' else x['nextspace']) 
                })
                n = n + 1
    
    return d_rooms

### Earthfloors要素をRoomsに展開

#### Outline

土間床を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [41]:
def integrate_earthfloors_to_rooms(region, d_earthfloors, d_rooms):
    
    n = 1
    for x in d_earthfloors :
        for y in d_rooms :
            if x['space'] == y['room_type'] :                
                y['Surface'].append({
                    'skin': True, 
                    'direction': 'Upward',
                    'floor': True,
                    'boundary': 'AnnualAverage',
                    'unsteady': True, 
                    'IsSoil': True,
                    'name': 'Earthfloor' + str(n), 
                    'area': x['area'],
                    'flr' : 0,
                    'Wall': {
                        'InHeatTrans': 1 / 0.15,
                        'Layers':[
                            {'name': 'RC',   'thick': 0.150, 'cond': 1.6, 'specH': 2000.0},
                            {'name': 'Soil', 'thick': 3.000, 'cond': 1.0, 'specH': 3300.0}
                        ]
                    },
                })
                n = n + 1
    
    return d_rooms

### 設備要素をRoomsに展開

#### Outline

設備の情報を室情報に紐付けする。

#### Function

In [42]:
def integrate_equipments_to_rooms(area_main, area_other, equipment_main, equipment_other, d_rooms):
    
    if area_main > 0 :
        for x in d_rooms :
            if x['room_type'] == 'main_occupant_room':
                x['equipment'] = equipment_main
    
    if area_other > 0 :
        for x in d_rooms :
            if x['room_type'] == 'other_occupant_room' :
                x['equipment'] = equipment_other
    
    return d_rooms

## 3. Convert

#### Function

In [43]:
def convert(d, equipment_main, equipment_other):
    
    common = d['common']
    ventilation = d['ventilation']
    envelope = d['envelope']
    
    region = common['region']
    
    d_calc_input = {}
    d_calc_input['common'] = make_common(common)
    d_calc_input['Rooms'] = get_rooms_initial(common, ventilation, envelope)
    
    d_calc_input['Rooms'] = integrate_walls_to_rooms(region, d['Walls'], d_calc_input['Rooms'])

    d_calc_input['Rooms'] = integrate_doors_to_rooms(region, d['Doors'], d_calc_input['Rooms'])

    d_calc_input['Rooms'] = integrate_windows_to_rooms(region, d['Windows'], d_calc_input['Rooms'])

    # 部位が無い場合の処理を後で追加する必要あり
    if ('Heatbridges' in d) == True :
        d_calc_input['Rooms'] = integrate_heatbridges_to_rooms(region, d['Heatbridges'], d_calc_input['Rooms'])

    d_calc_input['Rooms'] = integrate_earthfloorperimeters_to_rooms(region, d['EarthfloorPerimeters'],
                                                                        d_calc_input['Rooms'])
    d_calc_input['Rooms'] = integrate_earthfloors_to_rooms(region, d['Earthfloors'],
                                                               d_calc_input['Rooms'])

    d_calc_input['Rooms'] = integrate_innerwalls_to_rooms(region, d['InnerWalls'],
                                                              d_calc_input['Rooms'])

    d_calc_input['Rooms'] = integrate_equipments_to_rooms(d['common']['main_occupant_room_floor_area'],
                                                          d['common']['other_occupant_room_floor_area'],
                                                          equipment_main, equipment_other, d_calc_input['Rooms'])

    return d_calc_input

#### Example

In [44]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 120.0
    },
    'ventilation': {
        'air_change_rate' : 0.5,
    },
    'Walls': [
        {
            'name': 'Roof_main',
            'nextspace': 'OpenBackFloor',
            'direction': 'Top',
            'IsInContactWithOutsideAir': True, 
            'area': 16.95, 'space': 'main_occupant_room',
            'general_part_type': 'Roof',
            'structure': 'wood',
            'InputMethod' : 'InputAllDetails',
            'Parts': [{'AreaRatio': 1.0, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': None },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
            'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Roof_other', 'nextspace': 'OpenBackFloor', 'direction': 'Top', 'area': 16.95, 'space': 'other_occupant_room', 'general_part_type': 'Roof',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Roof_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'Top', 'area': 33.9, 'space': 'non_occupant_room', 'general_part_type': 'Roof',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Wall_SW_main', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 10.1575, 'space': 'main_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_other', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 10.1575, 'space': 'other_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 20.315, 'space': 'non_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_main', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 7.4575, 'space': 'main_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_other', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 7.4575, 'space': 'other_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 14.915, 'space': 'non_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_main', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 15.9725, 'space': 'main_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_other', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 15.9725, 'space': 'other_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 31.945, 'space': 'non_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_main', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 7.4275, 'space': 'main_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_other', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 7.4275, 'space': 'other_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 14.855, 'space': 'non_occupant_room', 'general_part_type': 'Wall',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Floor_main', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 15.0175, 'space': 'main_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_other', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 15.0175, 'space': 'other_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 30.035, 'space': 'non_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_main', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 1.1025, 'space': 'main_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_other', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 1.1025, 'space': 'other_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'Bottom', 'area': 2.205, 'space': 'non_occupant_room', 'general_part_type': 'Floor',
         'structure': 'wood', 'InputMethod' : 'InputUA', 'IsSunshadeInput': False, 'UA': 5.27}
    ],
    'Windows': [
        {'name': 'WindowSW_main', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 7.5625, 'space': 'main_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSW_other', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 7.5625, 'space': 'other_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSW_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'SW', 'area': 15.125, 'space': 'non_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51,  'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowNW_main', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 0.7925, 'space': 'main_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNW_other', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 0.7925, 'space': 'other_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNW_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 1.585, 'space': 'non_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_main', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 1.21, 'space': 'main_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None,
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_other', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 1.21, 'space': 'other_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 2.42, 'space': 'non_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowSE_main', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 1.4575, 'space': 'main_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSE_other', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 1.4575, 'space': 'other_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSE_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'SE', 'area': 2.915, 'space': 'non_occupant_room', 'UW': 3.49,
         'IsSunshadeInput': True, 'TypeWindow': 'Single', 'EtaInputMethod': 'InputValue', 'Eta': 0.51, 'TypeGlass': None, 
         'Y1': 0, 'Y2': 1.1, 'Z': 0.348}
    ],
    'Doors': [
        {'name': 'DoorNW_main', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 0.63, 'space': 'main_occupant_room', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_other', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 0.63, 'space': 'other_occupant_room', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NW', 'area': 1.26, 'space': 'non_occupant_room', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_main', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 0.54, 'space': 'main_occupant_room', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_other', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 0.54, 'space': 'other_occupant_room', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_nonliving', 'nextspace': 'OpenBackFloor', 'direction': 'NE', 'area': 1.08, 'space': 'non_occupant_room', 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'EarthfloorPerimeters': [
        {'nextspace': 'OpenBackFloor', 'direction': 'NW', 'length': 2.43, 'name': 'Entrance_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'nextspace': 'OpenBackFloor', 'direction': 'NE', 'length': 1.83, 'name': 'Entrance_NE', 'psi': 1.8, 'space': 'underfloor'},
        {'nextspace': 'OpenBackFloor', 'direction': 'OpenBackFloor', 'length': 4.25, 'name': 'Entrance_floor', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'name': 'Earthfloor', 'area': 3.24, 'space': 'underfloor'}
    ],
    'InnerWalls': [
        {'name': 'GroundFloor_main', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main_occupant_room', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_main', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'main_occupant_room'},
        {'name': 'GroundFloor_other', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other_occupant_room', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_other', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'other_occupant_room'},
        {'name': 'GroundFloor_nonliving', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 1.62,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'non_occupant_room', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_nonliving', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 1.62,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'non_occupant_room'},
        {'name': 'InnerFloor_main', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main_occupant_room', 'nextspace': 'other_occupant_room'},
        {'name': 'InnerFloor_main', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main_occupant_room', 'nextspace': 'non_occupant_room'},
        {'name': 'InnerFloor_other', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other_occupant_room', 'nextspace': 'main_occupant_room'},
        {'name': 'InnerFloor_other', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other_occupant_room', 'nextspace': 'non_occupant_room'},
        {'name': 'InnerFloor_nonliving', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'non_occupant_room', 'nextspace': 'main_occupant_room'},
        {'name': 'InnerFloor_nonliving', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'non_occupant_room', 'nextspace': 'other_occupant_room'},
        {'name': 'InnerCeiling_main', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main_occupant_room', 'nextspace': 'other_occupant_room'},
        {'name': 'InnerCeiling_main', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main_occupant_room', 'nextspace': 'non_occupant_room'},
        {'name': 'InnerCeiling_other', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 
                     720.0}],
         'space': 'other_occupant_room', 'nextspace': 'main_occupant_room'},
        {'name': 'InnerCeiling_other', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other_occupant_room', 'nextspace': 'non_occupant_room'},
        {'name': 'InnerCeiling_nonliving', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'non_occupant_room', 'nextspace': 'main_occupant_room'},
        {'name': 'InnerCeiling_nonliving', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'non_occupant_room', 'nextspace': 'other_occupant_room'}
    ]
}

equipment_main = {
    'cooling':{
        'main': {'q_rtd_c': 5600, 'q_max_c': 5944.619999999999, 'e_rtd_c': 3.2432},
        'sub': {}
    }, 
    'heating': {
        'main': {'q_rtd_h': 6685.3, 'q_max_h': 10047.047813999998, 'e_rtd_h': 4.157264},
        'sub': {}}
}

equipment_other = {
    'cooling': {
        'main': {'q_rtd_c': 5600, 'q_max_c': 5944.619999999999, 'e_rtd_c': 3.0576},
        'sub': {}
    },
    'heating': {
        'main': {'construct_area': 21.0}, 
        'sub': {'q_rtd_h': 6685.3, 'q_max_h': 10047.047813999998, 'e_rtd_h': 3.855424}
    }
}

print(convert(d, equipment_main, equipment_other))

KeyError: 'envelope'